In [1]:
%matplotlib inline

import os
import time
import imp
import cv2
import numpy as np
import tensorflow as tf

'''
from fast_rcnn.config import cfg
from datasets.mot import MOT
from fast_rcnn.net import FastRCNN
from fast_rcnn.roidb import prepare_roidb, add_bbox_regression_targets
from fast_rcnn.test import im_detect, vis_detections
from utils.cython_nms import nms
# from utils.visualize import *
'''

ImportError: Traceback (most recent call last):
  File "/home/yerong/local/Anaconda3/envs/py27/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/yerong/local/Anaconda3/envs/py27/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/yerong/local/Anaconda3/envs/py27/lib/python2.7/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
ImportError: libnvidia-fatbinaryloader.so.384.111: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/install_sources#common_installation_problems

for some common reasons and solutions.  Include the entire stack tr

In [ ]:
# Set up session
os.environ['CUDA_VISIBLE_DEVICES'] = str(3)
sess = tf.InteractiveSession()

# Define network
n_classes = 2
m = FastRCNN(n_classes)
tf.initialize_all_variables().run()

In [ ]:
# Load trained model
pretrained_model = 'yu_bbox_reg'
path_pretrained_model = \
    os.path.join('outputs', pretrained_model, 'model.ckpt')

saver = tf.train.Saver()
saver.restore(sess, path_pretrained_model)

In [ ]:
# Dataset configuration
train_videos = ['Venice-2',
                'KITTI-17',
                'KITTI-13',
                'ADL-Rundle-8',
                'ADL-Rundle-6',
                'ETH-Pedcross2',
                'ETH-Sunnyday',
                'ETH-Bahnhof',
                'PETS09-S2L1',
                'TUD-Campus',
                'TUD-Stadtmitte']
test_videos = ['Venice-1',
                'KITTI-19',
                'KITTI-16',
                'ADL-Rundle-3',
                'ADL-Rundle-1',
                'AVG-TownCentre',
                'ETH-Crossing',
                'ETH-Linthescher',
                'ETH-Jelmoli',
                'PETS09-S2L2',
                'TUD-Crossing']
videos = ['PETS09-S2L1']

In [ ]:
# Load the MOT dataset
videodb = MOT(videos)

prepare_roidb(videodb)
roidb = videodb.roidb
bbox_means, bbox_stds = add_bbox_regression_targets(roidb)
print('#videos: {}'.format(len(roidb)))
print('objects: {}'.format(videodb.objects))
print('bbox_means: {}'.format(bbox_means))
print('bbox_stds: {}'.format(bbox_stds))

In [ ]:
# Test
vi = 0
t = 50

im = cv2.imread(videodb.image_path_at(vi, t))
scores, boxes = im_detect(sess, m, im, roidb[vi][t]['boxes'],
                          videodb.im_scales[vi])

inds = np.where((scores[:, 1] > 0.0) &
                (roidb[vi][t]['gt_classes'] == 0))[0]
cls = 1
cls_scores = scores[inds, cls]
cls_boxes = boxes[inds, cls*4:(cls+1)*4]
all_boxes = np.hstack((cls_boxes, cls_scores[:, np.newaxis])) \
            .astype(np.float32, copy=False)
keep_inds = nms(all_boxes, 0.3)
vis_detections(im, videodb.classes[cls], all_boxes[keep_inds, :])